# Flyer Agent

Notes:
state_prime is next state

In [1]:
# !pip install tqdm
# ! pip install gym

In [2]:
import tensorflow as tf
import numpy as np
import random
import gym
import tqdm

import Flyer_Agent_Environment as FAE
slim = tf.contrib.slim

#%matplotlib inline 
%matplotlib notebook 

In [3]:
def calculate_naive_returns(rewards):
    """ Calculates a list of naive returns given a 
    list of rewards."""
    total_returns = np.zeros(len(rewards))
    total_return = 0.0
    for t in range(len(rewards), -1, -1): # changed end index to -1 RL
        total_return = total_return + rewards[t] # also fixed
        total_returns[t] = total_return
    return total_returns

In [4]:
def discount_rewards(rewards, gamma=0.99):
    discounted_returns = [0 for _ in rewards]
    discounted_returns[-1] = rewards[-1]
    for t in range(len(rewards)-2, -1, -1): # iterate backwards
        discounted_returns[t] = rewards[t] + discounted_returns[t+1]*gamma
    return discounted_returns

In [5]:
def epsilon_greedy_action(action_distribution, epsilon=1e-1):
    if random.random() < epsilon:
        return np.argmax(np.random.random(
           action_distribution.shape))
    else:
        return np.argmax(action_distribution)

In [6]:
def epsilon_greedy_action_annealed(action_distribution,
                                   episode,
                                   episode_stop,
                                   epsilon_start=1.0, 
                                   epsilon_end=1e-2):
    if (episode<episode_stop):
        annealed_epsilon = epsilon_start*(1.0-(episode/episode_stop)) + epsilon_end*(episode/episode_stop)
    else:
        annealed_epsilon = epsilon_end
    if random.random() < annealed_epsilon:
        return np.argmax(np.random.random(action_distribution.shape))
    else:
        return np.argmax(action_distribution)


In [7]:
[1,2][False] 

1

In [8]:
# this is about single episodes
class EpisodeHistory(object):

    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def add_to_history(self, state, action, reward, 
      state_prime):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.state_primes.append(state_prime)


In [9]:
# this holds multiple episodes
class Memory(object):

    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def reset_memory(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def add_episode(self, episode):
        self.states += episode.states
        self.actions += episode.actions
        self.rewards += episode.rewards
        self.discounted_returns += episode.discounted_returns

In [10]:
class PGAgent(object):

    def __init__(self, session, state_size, num_actions,
        hidden_size, learning_rate=1e-3):
        self.session = session
        self.state_size = state_size
        self.num_actions = num_actions
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        self.build_model()
        self.build_training()

    def build_model(self):
        with tf.variable_scope('pg-model'):
            self.state = tf.placeholder(
                shape=[None, self.state_size], 
                dtype=tf.float32)
            self.h0 = slim.fully_connected(self.state, 
            self.hidden_size)
            self.h1 = slim.fully_connected(self.h0, 
            self.hidden_size)
#             self.output = slim.fully_connected(
#                 self.h1, self.num_actions, 
#                 activation_fn=tf.nn.softmax)
#             # more hidden layers
            self.h2 = slim.fully_connected(self.h1, 
            self.hidden_size)
            self.h3 = slim.fully_connected(self.h2, 
            self.hidden_size)
            self.h4 = slim.fully_connected(self.h3, 
            self.hidden_size)
            self.output = slim.fully_connected(
                self.h4, self.num_actions, 
                activation_fn=tf.nn.softmax)

    def build_training(self):
        self.action_input = tf.placeholder(tf.int32, 
          shape=[None])
        self.reward_input = tf.placeholder(tf.float32, 
          shape=[None])

        # Select the logits related to the action taken
# this creates list of indices ?(ie num samples) long that are at start of each row in one-hot - then 
# adds indexes into each row of
# particular pre-selected action
        self.output_index_for_actions = (tf.range(
            0, tf.shape(self.output)[0]) * 
              tf.shape(self.output)[1]) + self.action_input
# then this flattens everything into 1-D, then selects from it using those indices
# all of this is a way to select only the neuron outputs that correspond to just the actions taken, rather
# than getting the values for all actions
        self.logits_for_actions = tf.gather(
            tf.reshape(self.output, [-1]), 
            self.output_index_for_actions)

        self.loss = -tf.reduce_mean(tf.log(tf.clip_by_value(self.logits_for_actions,1e-10,1.0)) * 
              self.reward_input)
# see book pg. 254
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate)
        self.train_step = self.optimizer.minimize(self.loss)

    def predict_action(self, 
                       state,
                       episode,
                       episode_stop,
                       epsilon_start=1.0,
                       epsilon_end=1e-2
        ):
        action_distribution = self.session.run(
            self.output, 
            feed_dict={self.state: [state]}
        )#[0]
#       print('action dist',action_distribution)
        action = epsilon_greedy_action_annealed(action_distribution,
                                                episode,
                                                episode_stop
        )
        return action
    
    def predict_action_0ep(self, 
                       state
        ):
        return np.argmax(self.session.run(
            self.output, 
            feed_dict={self.state: [state]}
        ))
    
    def show_current_policy(self):
        # Evaluate Current Policy - No Epsilon ------------------------------------
        pol_ep_hist = EpisodeHistory()
        state = env.reset()
        for n in range(20):    # figure out next action
            action = agent.predict_action_0ep(state)
            # take a step
            state_prime, reward, terminal, _ = env.step(action)
            state = state_prime
            pol_ep_hist.add_to_history(state, action, reward, state_prime)
        print('Policy Actions - No Epsilon')
        print(pol_ep_hist.actions)


In [11]:
# def main():
# Configure Settings
total_episodes = 2000
epsilon_start = 1.0
epsilon_end = 0.001
epsilon_stop_episode = 0.6*total_episodes#3000
BATCH_SIZE = 10 # this is the number of episodes generated before we train
MAX_EP_LENGTH = 199 #Fixed by env in Open AI Gym (we think)
render_start = False #-1
should_render = False # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
LEARNING_RATE = 1e-3
HIDDEN_SIZE = 16

# env = gym.make('CartPole-v0')
# state_size = env.observation_space.shape[0]  # 4 for CartPole-v0
# num_actions = env.action_space.n  # 2 for CartPole-v0
env = FAE.FlyerAgentEnv()
state_size = 4  
num_actions = 9

solved = False
batch_flag = False
num_episodes = 0

with tf.Session() as session:
    agent = PGAgent(session=session, state_size=state_size,
      num_actions=num_actions,
                    hidden_size=HIDDEN_SIZE, learning_rate = LEARNING_RATE)
    session.run(tf.global_variables_initializer())

    episode_rewards = []
    episode_terminal = [] # 1 for success, -1 for failure (o.b.)
    episode_steps = []
    episode_maxs = []
    batch_losses = []

    global_memory = Memory()
    steps = 0 # total number of steps in all episodes
    for i in tqdm.tqdm(range(total_episodes)): # multiple episode loop
        num_episodes += 1
        state = env.reset()
        episode_reward = 0.0 # cumulative sum of all rewards in episode
        episode_history = EpisodeHistory()
        epsilon_percentage = min(i/float(
          epsilon_stop_episode), 1.0)
        steps_this_ep = 0
        terminal = False
        j = 0 # num steps in episode
        while (j < MAX_EP_LENGTH) and (not terminal): # play single episode ------------------
        #for j in range(MAX_EP_LENGTH):
            j += 1
            # figure out next action
            action = agent.predict_action(state, 
                                          num_episodes,
                                          epsilon_stop_episode)
            # take a step
            # Note: action is not one-hot yet - it's a scalar
            state_prime, reward, terminal, _ = env.step(action)
            
            if (render_start > 0 and i > 
              render_start and should_render) \
                or (solved and should_render):
                env.render()
            episode_history.add_to_history(
                state, action, reward, state_prime)
            episode_reward += reward
            steps += 1
            steps_this_ep += 1
            # advance the state
            state = state_prime
            
#         print('number of steps in episode', j)  
        episode_history.discounted_returns = discount_rewards(episode_history.rewards)
        
#         # this is a modification that attempts to enhance the rewards for "good" episodes
#         # it doesn't seem to hurt - need more experimentation to know if it helps
#         for nn in range(len(episode_history.discounted_returns)):
#             episode_history.discounted_returns[nn] *= (1+(steps_this_ep/MAX_EP_LENGTH))**2

        global_memory.add_episode(episode_history)
        episode_rewards.append(episode_reward)
        # save sum of rewards, num_steps of this episode
        episode_steps.append(steps_this_ep)
        episode_maxs.append(max(episode_history.rewards))
        episode_terminal.append(terminal)

        # if we completed a minibatch then train
        if np.mod(i, BATCH_SIZE) == 0:
            agent.show_current_policy()
            print('global memory: ',global_memory.actions)
            feed_dict = {
            agent.reward_input: np.array(
              global_memory.discounted_returns),
            agent.action_input: np.array(
              global_memory.actions), 
            agent.state: np.array(
              global_memory.states)}
            _, batch_loss = session.run(
                [agent.train_step, agent.loss],
                  feed_dict=feed_dict)
            batch_losses.append(batch_loss)
            global_memory.reset_memory()            
            agent.show_current_policy()
            
#             if i > 1:
#                 batch_flag = True

#         #delete this
#         if terminal == 0 or batch_flag or num_episodes == 50:
#             print('terminal ',terminal,' batch_flag ',batch_flag)
#             break

#         if i % 10: # show results every 10th iteration
#             if np.mean(episode_rewards[:]) > 100.0:
#                 solved = True
#             else:
#                 solved = False
#     print('Solved:', solved, 'Mean Reward', np.mean(episode_rewards[:-100]))
    
    print('Num episodes:',num_episodes)
    
    print('Final Mean Reward - last 100', np.mean(episode_rewards[-100:-1]))
        
    print('Final Run - with Epsilon')
    print(episode_history.actions)
    
    
    print('Final Policy - No Epsilon')
    env.reset()
    e_p = 1.0 # so use end epsilon
    for n in range(20):    # figure out next action
        action = agent.predict_action_0ep(state)
        # take a step
        state_prime, reward, terminal, _ = env.step(action)
        state = state_prime
        print(state_prime,action,reward)  
        # NOTE: doesn't terminate with done flag

/home/johnnybravo/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
  1%|          | 11/2000 [00:00<00:40, 49.42it/s]

Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [6, 8, 6, 5, 0, 2, 6, 8, 3, 5, 5, 1, 6, 5, 4, 1, 6, 6, 4, 7, 4, 5, 3, 6, 2, 8, 2, 2, 5, 8, 7, 7, 7, 0, 3, 2, 2, 1, 1, 0, 2, 1, 2, 0, 5, 4, 2, 6, 6, 2, 4, 2, 1, 2, 4, 1, 7]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [0, 2, 3, 3, 7, 1, 8, 5, 5, 2, 6, 6, 6, 6, 0, 7, 6, 3, 2, 3, 3, 7, 6, 5, 6, 1, 8, 4, 6, 6, 4, 8, 0, 7, 8, 0, 2, 6, 3, 3, 1, 4, 8, 7, 4, 5, 0, 0, 8, 4, 0, 8, 6, 3, 0, 2, 2, 5, 0, 0, 1, 2, 2, 5, 7, 0, 4, 0, 3, 6, 2, 1, 0, 4, 1, 1, 7, 5, 7, 7, 2, 4, 6, 5, 3, 0, 5, 6, 5, 4, 6, 8, 8, 4, 4, 2, 8, 7, 8, 0, 3, 7, 5, 0, 2, 3, 1, 0, 8, 1, 4, 0, 1, 0, 3, 2, 4, 0, 0, 7, 8, 5, 6, 3, 5, 5, 4, 8, 0, 4, 7, 2, 2, 4, 3, 7, 8, 4, 8, 8, 6, 3, 5, 2, 2, 1, 7, 6, 2, 1, 6, 7, 1, 7, 2, 2, 6, 8, 6, 2, 3, 0, 3, 8, 8, 5, 0, 2, 7, 1, 6, 2, 0, 7, 0, 7, 

  1%|▏         | 28/2000 [00:00<00:31, 62.20it/s]

Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [6, 1, 3, 5, 1, 3, 3, 5, 7, 4, 3, 2, 2, 3, 0, 7, 6, 2, 8, 3, 5, 3, 7, 0, 4, 7, 1, 4, 5, 8, 7, 2, 7, 6, 6, 3, 2, 2, 0, 5, 8, 2, 5, 6, 2, 3, 0, 7, 1, 2, 1, 7, 0, 4, 4, 1, 4, 4, 1, 6, 4, 6, 3, 5, 7, 2, 1, 3, 0, 6, 4, 4, 0, 7, 4, 8, 3, 4, 1, 2, 5, 7, 6, 3, 7, 1, 3, 2, 8, 8, 1, 0, 4, 7, 7, 1, 8, 1, 6, 7, 5, 6, 8, 0, 5, 7, 1, 3, 3, 5, 6, 1, 6, 1, 6, 8, 3, 4, 8, 0, 6, 2, 5, 3, 3, 1, 0, 1, 3, 0, 4, 4, 7, 1, 2, 4, 7, 2, 2, 3, 6, 0, 5, 4, 4, 8, 7, 3, 6, 2, 5, 2, 2, 1, 4, 3, 8, 2, 5, 2, 8, 5, 7, 1, 6, 3, 2, 7, 1, 3, 0, 1, 8, 7, 4, 2, 4, 8, 4, 6, 0, 7, 0, 1, 6, 7, 1, 2, 4, 8, 2, 3, 4, 4, 5, 7, 5, 6, 8, 7, 3, 3, 2, 4, 8, 3, 6, 1, 2, 5, 6, 2, 6, 6, 0, 2, 6, 1, 7, 2, 8, 5, 7, 5, 5, 4, 5, 2, 2, 6, 3, 0, 1, 6, 7, 2, 3, 3, 8, 3, 1, 3, 5, 1, 1, 5, 6, 2, 2, 6, 4, 8, 6, 8, 5, 8, 3, 5, 3, 2, 0, 7, 8, 5, 6, 8, 3, 2, 1, 6, 0, 8, 6, 1, 0, 2, 4, 2, 4, 8, 4, 2, 4, 8]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2

  3%|▎         | 58/2000 [00:00<00:25, 75.58it/s]

...

 13%|█▎        | 251/2000 [00:03<00:24, 71.18it/s]

Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [1, 3, 3, 1, 3, 3, 3, 2, 5, 6, 5, 4, 1, 8, 8, 1, 5, 3, 7, 5, 3, 0, 5, 0, 6, 1, 8, 2, 6, 3, 5, 0, 1, 7, 2, 0, 2, 6, 4, 1, 0, 3, 5, 2, 0, 1, 5, 6, 7, 3, 4, 3, 2, 3, 8, 5, 2, 4, 0, 2, 2, 7, 0, 1, 4, 4, 3, 3, 1, 4, 8, 2, 5, 7, 6, 0, 2, 4, 4, 1, 4, 5, 4, 5, 5, 6, 2, 2, 5, 2, 2, 0, 3, 3, 5, 6, 6, 6, 1, 0, 2, 1, 7, 2, 0, 0, 6, 8, 5, 7, 2, 0, 2, 6, 1, 0, 3, 5, 4, 3, 3, 6, 2, 4, 4, 8, 2, 8, 5, 2, 7, 3, 2, 0, 3, 6, 7, 4, 3, 3, 8, 3, 2, 8, 2, 6, 7, 2, 6, 1, 7, 0, 2, 5, 5, 2, 2, 0, 2, 2, 3, 5, 2, 5, 2, 1, 1, 0, 0, 2, 8, 6, 3, 2, 2, 8, 1, 0, 3, 3, 2, 6, 2, 2, 1, 5, 3, 0, 3, 7, 4, 4, 0, 7, 8, 1, 7, 8, 4, 3, 0, 3, 6, 4, 6, 0, 3, 3, 4, 5, 5, 7, 3, 5, 4, 5, 1, 7, 5, 7]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [4, 2, 0, 6, 8, 2, 1, 4, 8, 0, 5, 8, 8, 

 14%|█▎        | 270/2000 [00:03<00:24, 71.69it/s]

Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
global memory:  [3, 4, 7, 2, 6, 3, 6, 4, 8, 2, 7, 0, 5, 2, 2, 4, 8, 1, 2, 4, 8, 5, 1, 3, 2, 3, 0, 5, 1, 4, 5, 5, 0, 7, 8, 2, 4, 4, 3, 6, 3, 2, 2, 7, 2, 2, 4, 0, 0, 7, 8, 8, 6, 1, 1, 1, 1, 0, 3, 5, 3, 2, 3, 2, 3, 2, 5, 7, 4, 3, 3, 6, 2, 5, 3, 8, 3, 0, 7, 0, 3, 5, 2, 5, 5, 8, 3, 2, 8, 3, 8, 5, 5, 2, 4, 5, 0, 4, 6, 2, 5, 6, 3, 2, 3, 7, 3, 4, 3, 1, 2, 4, 7, 0, 8, 2, 2, 5, 7, 7, 0, 1, 0, 1, 5, 7, 5, 8, 5, 7, 3, 2, 3, 7, 6, 8, 2, 7, 6, 1, 1, 3, 7, 6, 6, 1, 1, 2, 4, 4, 6, 4, 6, 0, 1, 2, 6, 1, 5, 7, 0, 5, 3, 3, 3, 6, 1, 4, 2, 0, 5, 1, 6, 4, 4, 4, 5, 4, 5, 7, 2, 7, 2, 4, 1, 7, 2, 3, 2, 3, 3, 6, 2, 2, 2, 2, 6, 8, 6, 1, 7, 1, 2, 5, 1, 1, 5, 1, 1, 6, 1, 8, 2, 7, 7, 0, 1, 1, 3, 2, 2, 0, 2, 6, 0, 2, 1, 5, 3, 5, 0, 8, 3, 2, 2, 8, 3, 3, 2, 5, 2, 0, 3, 6, 0, 3, 2, 2, 5, 8, 0, 2]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Policy Actions - No Epsilon
[2, 2, 2, 2, 2, 2, 2,

KeyboardInterrupt: 

In [ ]:
# main()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.plot(episode_rewards,'bd')

In [ ]:
    print('Final Mean Reward - last 100', np.mean(episode_rewards[-100:-1]))

In [ ]:
plt.figure()
plt.plot(episode_terminal)